In [1]:
#!pip install selenium
#!pip install geonamescache
from   helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from   bs4 import BeautifulSoup
from   IPython.display import clear_output #clears output
import datetime
import time

API_Key = 'KEY'
API_Key_Secret = 'KEY'

Bearer_Token = 'KEY'

Access_Token = 'KEY'
Access_Token_Secret = 'KEY'

In [15]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 

### retrieving the tweets
we will begin by saving the WOEIDs of each country in variables(The WOEID is a 32 bit adress of a place on earth, it's an odd system twitter adopted for some reason)

we will then ask the API to get us 2000 tweets from each WOEID.
we do this by giving the API a spacial query:
"plase:" + WOEID = tells the API to only retrieve tweets from that location

"-filter:replies" = tells the API to ignore tweets that are actually replies
we ignore them since replies rarely get retweets and they basically filled our DataFrame with a lot of replies that have 0 retweets.

"-filter:retweets" = we removed this since apperantly whenever someone retweets a tweets, it's concidered by twitter as it's own tweet. so what ended up happening is that we had several of the same tweet show up, it filled our Data Frame with duplicated.

afterwards we activate a function we wrote called "getAllTrends()"
the function uses the lenium to crawl into 'www.trends24.in' and get the trending topics in each country and every city in it(that is avalible)

twitter's API has a function that allows us to get the trends in each area, however it has a very limited rate. So we used a third party website instead.

In [3]:
'''
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
'''

# Search for tweets within the target countries
usa = '96683cc9126741d1'
canada = "3376992a082d67c7"
great_britain = '6416b8512febefc9'
Australia = '3f14ce28dc7c4566'
new_zealand = "0b039ce18ddf1cb9"

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
max_count_4_each_country = 2000


#add usa
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ usa + ' -filter:replies -filter:retweets',
                             tweet_mode="extended",
                             include_entities=True,count = 100).items(max_count_4_each_country)

print('Us secured')
time.sleep(10)

#add Canada
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ canada + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('canada secured')
time.sleep(10)

#add "Great" Britian
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ great_britain + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('UK secured')
time.sleep(10)

#add Australia
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ Australia + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('kengooro land secured')
time.sleep(10)

#add new zealand
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ new_zealand + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print("kengooroon't land secured")


'''
#return this if you are using pages again!!!!!
for page in ALL_pages:
    tweets_list.extend(page)
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]    
'''

trends = getAllTrends()#get trends

print('done')

Us secured
canada secured
UK secured
kengooro land secured
kengooroon't land secured
done


In [4]:
tweets_list = [t for t in tweets_list]
len(tweets_list)

10000

### crawling
now the interseting part.

firstly we will extract everything we need from each tweet
1) the text
2) the amount of words
3) how many likes the tweet has
4) how many retweets the tweet has
5) is it a quote tweet
6) the date the tweet was posted
7) the ID of the tweet(this will be useful in the reextraction)
8) the user's information

then we will go over all users we have:
because the twitter API likes to hide literally anything useful we will have to extract some information with brute force
so we will use selenium to enter each users profile page

using selenium and the API we extract he following:
1) the screen name of the user(the name that is eunique to that spacific user)
2) how many followers the user has
3) how many people the user follows
4) when the user joined(account age)
5) how many tweets the user has tweeted in general
6) is the user verified
7) gender

the API didn't give us the gender and age of the user.
so we crawled into their profile and extracted it.

in twitter it is very common to put the user's pronounce in their name,bio and or even location
so we searched there


In [5]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
#location = [];

city = [];
country = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
acc_age = [] 
respectability = []
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
hashtags_count = []
trending_for_area = []
is_using_trend = []
users = [];

print('loading users')

arr = []
i = 1

for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    
    word_count.append(countWordsInString(t.full_text))
    
    likes.append( t.favorite_count)
    
    retweet_count.append(t.retweet_count)
    
    is_quote_status.append(t.is_quote_status)
    
    date.append(t.created_at.isoformat()[:10])
    
    tweet_id.append(t.id)
    
    
        
    city.append(t.place.name)
    country.append( t.place.country)
    
    loc_trends = getTrendsByLoc(t.place.name,t.place.country,trends)
    trending_for_area.append(loc_trends)
    is_using_trend.append(areTrendUsed(t.full_text,loc_trends))
    
    ht = [t['text'] for t in t.entities['hashtags']]
    ht_count = len(ht)
    
    if ht == []:
        ht = np.nan
        ht_count = 0
        
    hashtags.append(ht)
    hashtags_count.append(ht_count)
    
    
    
    
    
        
for x in users:
    if i%26 == 0:
        print("waiting")
        time.sleep(6)
    j = x._json
    
    #opens the page
    screen_name = j['screen_name']    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count'])
    followers.append(j['followers_count'])
    
    if(j['followers_count'] != 0):
        respectability.append(float(j['friends_count']) / float(j['followers_count']))
    else:
        respectability.append(0)
        
    joined.append(x.created_at.isoformat()[:10])
    acc_age.append(2023 - x.created_at.year + float(x.created_at.day)/12)
    tweet_count_user.append(x.statuses_count)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']))
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

10001
done


In [6]:
max = len(gender)

for i in range(0,max):#rounds the account ages
    acc_age[i] = round(acc_age[i],1)

In [7]:
metaData=['--------' for i in range(max)]
dick = {"name":user_name[:max],
        "age":age[:max],
        'city':city[:max],
        'country':country[:max],
        "gender":gender[:max],
        'acount age':acc_age[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'respectability':respectability[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'word count': word_count[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'hashtag count':hashtags_count[:max],
        'trending': trending_for_area[:max],
        'using trends?':is_using_trend[:max],
        'metaData':metaData[:max],
        'join_date':joined[:max],
        'date':date[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [8]:
df

,name,age,city,country,gender,acount age,total tweets,followers,following,respectability,...,word count,is quote,hashtags,hashtag count,trending,using trends?,metaData,join_date,date,tweet id
0,giathxo,NaN,Bethpage,United States,NaN,11.3,3867,216,445,2.060185,...,5,True,NaN,0,"[Saka, #AVLARS, Zinchenko, Xhaka, #SaturdayMor...",False,--------,2013-12-16,2023-02-18,1626942985463013376
1,chenguanxi7979,NaN,Los Angeles,United States,NaN,2.5,3,1,3,3.000000,...,2,False,NaN,0,"[#gokingsgo, #AVLARS, Saka, Zinchenko, Mings, ...",False,--------,2022-09-18,2023-02-18,1626942982774525954
2,HolaArizona,NaN,Phoenix,United States,NaN,11.7,3929,68,163,2.397059,...,57,False,NaN,0,"[#SmackDown, Saka, #AVLARS, Zinchenko, Xhaka, ...",False,--------,2013-12-20,2023-02-18,1626942982397267969
3,420PandaNation,NaN,Michigan,United States,NaN,3.1,5502,2712,2478,0.913717,...,19,False,NaN,0,"[Saka, #AVLARS, Zinchenko, Xhaka, #SaturdayMor...",False,--------,2022-09-25,2023-02-18,1626942978760613889
4,JeffreyLuscombe,NaN,Fort Lauderdale,United States,NaN,15.5,342636,13792,11222,0.813660,...,16,False,NaN,0,"[Saka, #AVLARS, Zinchenko, Xhaka, #SaturdayMor...",False,--------,2008-11-06,2023-02-18,1626942973282770945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,LuisaLongone,NaN,Auckland,New Zealand,NaN,10.9,1518,487,824,1.691992,...,8,True,"[MIEExpertNZ, MIEE, MIEEFellow]",3,"[#NZvENG, MySpace, napier, SMS 2FA, Chantelle,...",False,--------,2014-06-23,2023-02-16,1626295677088329728
9996,moanaduffy25,NaN,Auckland,New Zealand,NaN,3.4,127,2,12,6.000000,...,23,False,NaN,0,"[#NZvENG, MySpace, napier, SMS 2FA, Chantelle,...",False,--------,2022-07-29,2023-02-16,1626295570699780096
9997,everylotchc,NaN,Christchurch City,New Zealand,NaN,4.4,33456,564,59,0.104610,...,7,False,NaN,0,"[#NZvENG, MySpace, napier, SMS 2FA, Chantelle,...",False,--------,2020-01-17,2023-02-16,1626295394639683584
9998,everylotakl,NaN,Auckland,New Zealand,NaN,4.3,38084,828,55,0.066425,...,5,False,NaN,0,"[#NZvENG, MySpace, napier, SMS 2FA, Chantelle,...",False,--------,2020-01-16,2023-02-16,1626295393003933697


The crawling is finished, we can close the browser and save the DataFrame to use later

In [9]:
DRIVER.quit()

In [16]:
df.to_csv('data7.csv')